
# 🧪 07 — MCP Projects Lab (Complete Project Plans, Option A)

This notebook is your **MCP Projects Lab**:

- A set of **end-to-end project blueprints** you can implement
- Each project is:
  - scoped
  - structured
  - connected to the rest of this repo (TOC, roadmap, tools)
- Designed as **portfolio-ready** and **team-teachable**

You can:

- Implement them 1-by-1
- Use them as templates for your own domains
- Turn them into blog series, talks, or internal workshops



## 0. How to Use This Lab

For each project:

- There is a **consistent structure**:
  - Overview
  - User stories
  - Architecture
  - Tools & resources
  - Security
  - Phased implementation plan
  - Stretch goals

Suggested ways to use:

- Pick **one project per month** and implement it
- Use sections as **Jira epics + stories**
- Turn each project into:
  - a GitHub repo
  - docs in `docs/`
  - a section in your portfolio / CV

The projects are designed to **connect with your RAG world-class repo**, but they are also useful alone.



## Project 1 — Internal Knowledge & Docs MCP (RAG-Ready)

### 1.1 Overview

Build an MCP server that exposes your **internal knowledge base**:

- Markdown / docs folders
- Project READMEs
- Internal handbook pages

LLMs and agents can:

- search documents
- retrieve content
- inspect file metadata

This is the **bridge** between your filesystem/docs and your AI assistants.

### 1.2 Primary User Stories

- *As a developer*, I want to ask:
  - “Where is the onboarding doc for Project X?”  
- *As a new hire*, I want to ask:
  - “What services are involved in our payment flow?”  
- *As an SRE*, I want to ask:
  - “Show me the runbooks related to database outages.”  

### 1.3 Architecture (High-Level)

Components:

- **MCP Server: `MCP_Docs`**
  - Filesystem tools:
    - `listFiles(path, recursive?, max_items?)`
    - `readFile(path, max_bytes?)`
    - optional: `getFileMetadata(path)`  
  - RAG tools (optional, integration with RAG repo):
    - `searchDocs(query, tags?, top_k?)`
    - `getDocById(id)`  

Data:

- `docs_root/` base folder (Markdown / text)
- Optional vector index (if you plug in RAG)

### 1.4 Tools & Resources

Tools (MCP):

- `list_docs`
- `read_doc`
- `search_docs` (if RAG)
- `summarize_doc` (optional, calls LLM with context)

Resources (conceptual):

- Document handles:
  - `doc://path/to/file.md`
  - or internal IDs for RAG index items  

### 1.5 Security & Safety

- Sandbox `docs_root`
- Exclude:
  - secrets files
  - config with keys
  - .git, node_modules, etc.  
- Access:
  - Can be read-only for most use cases
  - Optional tenant-based directory prefixes

### 1.6 Implementation Phases

**Phase 1 — Filesystem-only MVP**

- Implement tools:
  - `list_docs(path)`
  - `read_doc(path, max_bytes)`  
- Connect base directory:
  - `BASE_DIR = /path/to/docs_root`  

**Phase 2 — Metadata & Search Index (Local)**

- Maintain a simple index:
  - file paths
  - titles (from first heading)
  - tags (from frontmatter / folders)  
- Add:
  - `search_docs_by_title(query, limit)`  

**Phase 3 — RAG Integration**

- Connect to:
  - your RAG repo’s indexing pipeline  
- Add:
  - `search_docs(query, top_k)` calling vector DB
  - `get_doc_by_id(id)` returning text + metadata  

### 1.7 Stretch Goals

- Multi-language docs support  
- Git revision awareness (show doc version as of a given commit)  
- “What changed recently?” tools for changed files  



## Project 2 — RAG Support MCP (Knowledge + Tools)

### 2.1 Overview

This project builds a dedicated MCP server to support **RAG applications**.

Instead of hardcoding RAG logic in each app, you centralize it:

- Ingestion
- Index management
- Retrieval
- Context shaping

### 2.2 Primary User Stories

- *As an AI engineer*, I want:
  - one MCP endpoint to call for `searchKnowledgeBase`  
- *As a product team*, we want:
  - consistent retrieval behavior across assistants  
- *As a platform team*, we want:
  - observability on retrieval performance  

### 2.3 Architecture

Components:

- **MCP Server: `MCP_RAG`**
  - Connects to:
    - vector DB (Chroma, pgvector, Pinecone, etc.)
    - metadata store (Postgres, etc.)  

Key tools:

- `indexDocuments(source, options?)`
- `searchKnowledgeBase(query, top_k, filters?)`
- `getDocumentById(id)`
- `getChunksByIds(ids[])`

### 2.4 Tools & APIs

MCP tools:

- `searchKnowledgeBase`
  - Inputs:
    - `query: string`
    - `top_k?: integer`
    - `filters?: object`  
  - Outputs:
    - `results: [{ id, title, score, snippet }]`
    - `total`
    - `truncated`

- `getDocumentById`
  - Inputs: `id: string`
  - Output:
    - `id`, `title`, `full_text`, `metadata`  

### 2.5 Security & Governance

- Access policies per collection:
  - public vs internal vs restricted  
- Control which **tenants** can see which docs  
- Logging:
  - which queries are run
  - which docs are retrieved  

### 2.6 Implementation Phases

**Phase 1 — Single Collection, Local Vector DB**

- Index:
  - sample docs
- Implement:
  - `searchKnowledgeBase`  
- Integrate with RAG notebook from your other repo.

**Phase 2 — Multi-Collection, Filters**

- Add:
  - `collection` param
  - metadata filters (e.g., `department`, `tag`)

**Phase 3 — Quality & Evaluation Hooks**

- Add:
  - `logRetrievalEvent(query, docs, scores)`
- Integrate with:
  - evaluation notebooks from your RAG repo (RAGAS, LLM-as-judge, etc.)  

### 2.7 Stretch Goals

- On-demand re-indexing tools  
- Tools to run **eval jobs** via MCP:
  - `runRetrievalEval(dataset_id)`  



## Project 3 — DevOps & SRE MCP (Logs + Metrics + Incidents)

### 3.1 Overview

Build an MCP server that provides **safe, structured access** to:

- logs
- metrics
- incidents / alerts

This can power:

- “SRE Copilot”
- on-call assistants
- incident review tooling

### 3.2 Primary User Stories

- *As an on-call engineer*, I want to ask:
  - “Show me errors for service X in the last 15 minutes.”
- *As an SRE*, I want:
  - “Summarize metrics for latency and errors for service Y today.”
- *As an incident commander*, I want:
  - “What previous incidents look similar to this one?”

### 3.3 Architecture

Components:

- **MCP Server: `MCP_DevOps`**
  - Connectors to:
    - log store (e.g., Elasticsearch, Loki, CloudWatch)
    - metrics store (e.g., Prometheus, Datadog)
    - incident tracker (e.g., Jira, PagerDuty, internal DB)

Tools:

- Logs:
  - `getRecentLogs(service, window, limit?)`
- Metrics:
  - `getMetricSnapshot(metric_name, service, window)`
- Incidents:
  - `listIncidents(service, window)`
  - `getIncidentById(id)`

### 3.4 Safety & Limits

- Enforce max log volume per call:
  - `max_bytes`
  - `max_lines`  
- Restrict:
  - which services can be queried (allow-list)
  - which environments (dev/stage/prod usage policies)  

### 3.5 Implementation Phases

**Phase 1 — Static/Mock Data**

- Implement tools with:
  - static JSON files as sources  
- Useful for:
  - prototyping flows
  - agent behavior testing  

**Phase 2 — Real Log & Metrics Integration**

- Connect to one real system (e.g., Prometheus or Loki)  
- Implement:
  - careful query templates
  - truncation
  - error mapping  

**Phase 3 — Incident Data & Summaries**

- Add tools:
  - `getIncidentOverview(id)` – combinational tool that calls others  
- Provide structured summaries:
  - timelines
  - related services
  - follow-up actions  

### 3.6 Stretch Goals

- On-call runbook integration (link to docs MCP)  
- Slack/ChatOps hooks around MCP DevOps tools  



## Project 4 — Finance Learning & Planning MCP (Aligned with AI Finance Assistant)

### 4.1 Overview

This project connects to your **AI Finance Assistant** idea:

- Build an MCP server that offers:
  - curated financial knowledge access
  - simulated portfolios
  - planning tools

The **AI Finance Assistant** then uses these tools through MCP.

### 4.2 Primary User Stories

- *As a learner*, I want:
  - “Explain compound interest using my numbers.”  
- *As a beginner investor*, I want:
  - “If I invest X per month, what could this grow to in 20 years?”  
- *As a portfolio learner*, I want:
  - “Simulate what would happen if I shift 10% from bonds to stocks.”  

### 4.3 Architecture

Components:

- **MCP Server: `MCP_FinanceLab`**
  - Knowledge tools:
    - `getConceptExplainer(concept_id)` – returns curated explanation text  
    - `listConcepts(topic)`  
  - Planning tools:
    - `simulateSavingsPlan(monthly_amount, years, rate?)`
    - `simulatePortfolioChange(portfolio_state, change)`  

- Data:
  - curated educational content (e.g., JSON files, PDFs processed via RAG)
  - simple portfolio models (not real trading)

### 4.4 Tools & Resources

Tools:

- `getConceptExplainer`
- `listConcepts`
- `simulateSavingsPlan`
- `simulatePortfolioScenario`

Resources:

- concept IDs:
  - `finance_concept://compound_interest`
  - `finance_concept://index_funds`  

### 4.5 Safety & Disclaimers

- Content must emphasize:
  - educational use
  - not financial advice  
- Tools should:
  - avoid linking to real brokerage accounts
  - operate on **simulated** or demo data only  

### 4.6 Implementation Phases

**Phase 1 — Concept Library Only**

- Implement:
  - `listConcepts(topic)`
  - `getConceptExplainer(concept_id)`  
- Backed by markdown/JSON educational texts.

**Phase 2 — Simple Simulators**

- Add:
  - `simulateSavingsPlan` (pure math)
  - return table of:
    - year
    - contributions
    - balance  

**Phase 3 — Portfolio Scenarios**

- Add:
  - `simulatePortfolioScenario` with:
    - baseline allocations
    - simple projected returns  

### 4.7 Stretch Goals

- Connect to RAG knowledge base (RAG MCP project) for deeper context  
- Multi-language concept explanations  



## Project 5 — Education & Course MCP (Curriculum + Progress)

### 5.1 Overview

Build an MCP server that:

- Exposes a **course catalog** and **learning plans** (for MCP, RAG, etc.)
- Provides tools for:
  - listing modules
  - fetching lessons
  - tracking progress (conceptually)

This can power a **study assistant** that knows your curriculum.

### 5.2 User Stories

- *As a learner*, I want:
  - “What should I study this week for MCP?”  
- *As someone designing courses*, I want:
  - “Generate a weekly plan for this student’s level.”  

### 5.3 Architecture

Components:

- **MCP Server: `MCP_Edu`**
  - Data:
    - “course graph”:
      - topics
      - modules
      - lessons  
  - Tools:
    - `listCourses()`
    - `getCourseOutline(course_id)`
    - `getLessonContent(lesson_id)`
    - `suggestNextLessons(current_state)`  

### 5.4 Tools & Resources

Tools:

- `listCourses`
- `getCourseOutline`
- `getLessonContent`
- `suggestNextLessons`

Resources:

- Course IDs:
  - `course://world-class-mcp-foundation`
  - `course://world-class-rag-foundation`  

### 5.5 Implementation Phases

**Phase 1 — Static Curriculum (MCP & RAG)**

- Encode:
  - your MCP roadmap
  - your RAG roadmap
- Implement:
  - `listCourses`
  - `getCourseOutline`
  - `getLessonContent`  

**Phase 2 — Simple Progress Tracking**

- Store progress state as:
  - JSON per user (for personal use)
- Add tool:
  - `markLessonCompleted(lesson_id)` (for local/test use)

**Phase 3 — Personalized Suggestions**

- Use:
  - `suggestNextLessons(current_state)`
- Combine:
  - prerequisites
  - difficulty
  - learner’s goals  

### 5.6 Stretch Goals

- Generate personalized study plans using LLM + MCP_Edu tools  
- Integrate with calendar (schedule sessions)  



## 6. Cross-Project Connections

These projects are meant to **interlock**:

- `MCP_Docs` (Project 1) and `MCP_RAG` (Project 2):
  - RAG uses docs as source  
- `MCP_DevOps` (Project 3) and `MCP_Docs`:
  - runbooks in docs, incidents referencing them  
- `MCP_FinanceLab` (Project 4) and your AI Finance Assistant:
  - MCP server is the backend for financial tools  
- `MCP_Edu` (Project 5) and this entire repo:
  - curriculum content comes from your own notebooks & docs  

You can build a **meta-agent** that uses:

- `MCP_Edu` to understand your learning roadmap  
- `MCP_RAG` to access technical references  
- `MCP_Docs` to browse local docs  
- `MCP_DevOps` or `MCP_FinanceLab` for domain questions  



## 7. Portfolio & Storytelling

For each project, capture:

- **Problem statement** (business + technical)  
- **Architecture diagram** (even simple)  
- **Key MCP tools**: names + schemas  
- **Security notes**: how you kept it safe  
- **Observability**: what you log/measure  

Turn them into:

- GitHub READMEs  
- Blog posts:
  - “How I built an MCP DevOps copilot”
  - “World-class RAG via MCP tools”
- Talks for meetups or internal tech talks  

The combination of:

- RAG repo + MCP repo + Finance Assistant + education MCP

gives you a **world-class, multi-project portfolio** that showcases:

- MCP
- RAG
- agents
- backend design
- security & observability thinking  
